In [1]:
# Imports 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os


# Directories
# Stage Dir
stage_1_dir = "stage-1"

# Code Dir
code_dir = "code"

# Data Dir
data_dir = "data"

# Filenames
churn_od_txt = "Churn_OD.txt"
churn_users_pf_txt = "Churn_UsersProfile.txt"

# "freguesias-metadata.xlsx"
# Source: https://dados.gov.pt/pt/datasets/freguesias-de-portugal/
pt_parish_metadata = "freguesias-metadata.xlsx"

# "DistritosConcelhosFreguesias_CAOP2013_Populacao_Censos2011.xls"
# Source: https://www.factorvirtual.com/blog/distritos-concelhos-e-freguesias-de-portugal
dicofre_den_pop = "DistritosConcelhosFreguesias_CAOP2013_Populacao_Censos2011.xls"

# "pordata_densidade_populacional.xlsx"
# Source: https://www.pordata.pt/Subtema/Municipios/Censos+da+População-204
pordata_dens_pop = "pordata_densidade_populacional.xlsx"

In [2]:
# Load Churn OD data
churn_od = pd.read_csv(
    filepath_or_buffer = os.path.join(stage_1_dir, data_dir, churn_od_txt),
    delimiter = "|",
    encoding = 'utf-8'
)

# Get churn_od head for debugging purposes
churn_od.head()

,Region_of_Origin,District_of_Origin,County_of_Origin,Region_of_Public_Transportation,District_of_Public_Transportation,County_of_Public_Transportation,Dicofre_ParishCode_of_Public_Transportation,Demand_weight
0,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110608,0.307323
1,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110639,0.069997
2,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110658,0.066059
3,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110654,0.059847
4,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110633,0.052341


In [3]:
# Load Churn Users Profile data
churn_users_pf = pd.read_csv(
    filepath_or_buffer = os.path.join(stage_1_dir, data_dir, churn_users_pf_txt),
    delimiter = '|',
    encoding = 'utf-8'
)


# Debugging purposes
churn_users_pf.head()

,Region_of_Origin,District_of_Origin,County_of_Origin,Period,GenderDescription,AgeClassDescription,Average_BusUsers_per_Day
0,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,15-24,294.194206
1,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,25-34,1081.652817
2,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,35-44,235.836653
3,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,45-54,840.951323
4,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,55-64,1216.148092


In [4]:
# Load Parish Metadata
parish_metadata = pd.read_excel(os.path.join(stage_1_dir, data_dir, pt_parish_metadata), encoding = 'utf-8')
parish_metadata = parish_metadata[["distrito", "concelho", "freguesia", "dicofre"]]

# Debugging purposes
print(parish_metadata.shape)
parish_metadata.head()

(3092, 4)


,distrito,concelho,freguesia,dicofre
0,Aveiro,Águeda,Aguada de Cima,010103
1,Aveiro,Águeda,Fermentelos,010109
2,Aveiro,Águeda,Macinhata do Vouga,010112
3,Aveiro,Águeda,Valongo do Vouga,010119
4,Aveiro,Águeda,União das freguesias de Águeda e Borralha,010121


In [5]:
# Load Population Density
population_density_censos = pd.read_excel(os.path.join(stage_1_dir, data_dir, dicofre_den_pop), encoding = 'unicode_escape')

population_density_censos.rename(columns={"Freguesia (FR)":"dicofre", "PopRes_2011 (nº)":"População"}, inplace=True)
population_density_censos["dicofre"].str.strip()

# Debugging purposes
print(population_density_censos.shape)
population_density_censos.head()

(2882, 9)


,Distrito (DT),Designação DT,Concelho (CC),Designação CC,dicofre,Designação FR,População,Rural,Litorâneo
0,1,Aveiro,101,Águeda,10103,Aguada de Cima,4013,S,NaN
1,1,Aveiro,101,Águeda,10109,Fermentelos,3258,S,NaN
2,1,Aveiro,101,Águeda,10112,Macinhata do Vouga,3406,S,NaN
3,1,Aveiro,101,Águeda,10119,Valongo do Vouga,4877,S,NaN
4,1,Aveiro,101,Águeda,10121,União das freguesias de Águeda e Borralha,13576,S,NaN


In [6]:
# Join parish metadata and this by dicofre
population_density_censos_dicofres = population_density_censos.merge(parish_metadata, on=["dicofre"], how="left")
population_density_censos_dicofres = population_density_censos_dicofres[["Designação DT", "Designação CC", "dicofre", "Designação FR", "População"]]
population_density_censos_dicofres.rename(columns={"Designação DT":"Distrito", "Designação CC":"Concelho", "Designação FR":"Freguesia"}, inplace=True)
population_density_censos_dicofres.head()

,Distrito,Concelho,dicofre,Freguesia,População
0,Aveiro,Águeda,10103,Aguada de Cima,4013
1,Aveiro,Águeda,10109,Fermentelos,3258
2,Aveiro,Águeda,10112,Macinhata do Vouga,3406
3,Aveiro,Águeda,10119,Valongo do Vouga,4877
4,Aveiro,Águeda,10121,União das freguesias de Águeda e Borralha,13576


In [7]:
# We only want the district, county and parish and the codes, along with the population density
pop_density = population_density_censos_dicofres.copy()

# Debugging purposes
pop_density.head()

,Distrito,Concelho,dicofre,Freguesia,População
0,Aveiro,Águeda,10103,Aguada de Cima,4013
1,Aveiro,Águeda,10109,Fermentelos,3258
2,Aveiro,Águeda,10112,Macinhata do Vouga,3406
3,Aveiro,Águeda,10119,Valongo do Vouga,4877
4,Aveiro,Águeda,10121,União das freguesias de Águeda e Borralha,13576


In [9]:
# Population density by district and county
# District
pop_density_district = pop_density.copy().groupby("Distrito").sum().reset_index(drop=False)

# Debugging purposes
pop_density_district.head()

,Distrito,População
0,Aveiro,714197
1,Beja,152758
2,Braga,848185
3,Bragança,136252
4,Castelo Branco,196264


In [10]:
# County
pop_density_county = pop_density.copy().groupby(["Distrito", "Concelho"]).sum().reset_index(drop=False)

# Debugging purposes
pop_density_county.head()

,Distrito,Concelho,População
0,Aveiro,Albergaria-a-Velha,25252
1,Aveiro,Anadia,29150
2,Aveiro,Arouca,22359
3,Aveiro,Aveiro,78450
4,Aveiro,Castelo de Paiva,16733


In [11]:
# Rename columns to help in the join
population_density_censos_dicofres_renamed = population_density_censos_dicofres.copy()
population_density_censos_dicofres_renamed.rename(columns = {'dicofre': 'Dicofre_ParishCode_of_Public_Transportation'}, inplace=True)
population_density_censos_dicofres_renamed.head()

,Distrito,Concelho,Dicofre_ParishCode_of_Public_Transportation,Freguesia,População
0,Aveiro,Águeda,10103,Aguada de Cima,4013
1,Aveiro,Águeda,10109,Fermentelos,3258
2,Aveiro,Águeda,10112,Macinhata do Vouga,3406
3,Aveiro,Águeda,10119,Valongo do Vouga,4877
4,Aveiro,Águeda,10121,União das freguesias de Águeda e Borralha,13576


In [12]:
# Merge churn_od with pop_density_censos_renamed
churn_od_pop_density = churn_od.merge(
    population_density_censos_dicofres_renamed,
    on=["Dicofre_ParishCode_of_Public_Transportation"],
    how="left"
)

# Debugging
print(churn_od_pop_density.shape)
churn_od_pop_density.head()

(2253, 12)


,Region_of_Origin,District_of_Origin,County_of_Origin,Region_of_Public_Transportation,District_of_Public_Transportation,County_of_Public_Transportation,Dicofre_ParishCode_of_Public_Transportation,Demand_weight,Distrito,Concelho,Freguesia,População
0,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110608,0.307323,Lisboa,Lisboa,Benfica,36985
1,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110639,0.069997,Lisboa,Lisboa,São Domingos de Benfica,33043
2,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110658,0.066059,Lisboa,Lisboa,Belém,16528
3,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110654,0.059847,Lisboa,Lisboa,Alvalade,31813
4,R1 - AM Lisboa,Lisboa,Amadora,R1 - AM Lisboa,LISBOA,LISBOA,110633,0.052341,Lisboa,Lisboa,Olivais,33788


In [13]:
# Get AvgUsersPerDay by District and County of Origin
churn_users_grouped  = churn_users_pf.copy()[["Region_of_Origin", "District_of_Origin", "County_of_Origin", "Period", "Average_BusUsers_per_Day"]]
churn_users_grouped = churn_users_grouped.groupby(["Region_of_Origin", "District_of_Origin", "County_of_Origin", "Period"]).sum().reset_index(drop=False)
churn_users_grouped.head()

,Region_of_Origin,District_of_Origin,County_of_Origin,Period,Average_BusUsers_per_Day
0,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,8474.874368
1,R1 - AM Lisboa,Lisboa,Amadora,Sep-20 to Jan-21,6985.896633
2,R1 - AM Lisboa,Lisboa,Cascais,Sep-19 to Feb-20,13023.810109
3,R1 - AM Lisboa,Lisboa,Cascais,Sep-20 to Jan-21,6693.154115
4,R1 - AM Lisboa,Lisboa,Lisboa,Sep-19 to Feb-20,27874.289765


In [14]:
# Get AvgUsersPerDay by District of Origin
churn_users_grouped_district  = churn_users_pf.copy()[["Region_of_Origin", "District_of_Origin", "Period", "Average_BusUsers_per_Day"]]
churn_users_grouped_district = churn_users_grouped.groupby(["Region_of_Origin", "District_of_Origin", "Period"]).sum().reset_index(drop=False)
churn_users_grouped_district.head()

,Region_of_Origin,District_of_Origin,Period,Average_BusUsers_per_Day
0,R1 - AM Lisboa,Lisboa,Sep-19 to Feb-20,122085.933172
1,R1 - AM Lisboa,Lisboa,Sep-20 to Jan-21,81057.911693
2,R1 - AM Lisboa,Setúbal,Sep-19 to Feb-20,41852.012153
3,R1 - AM Lisboa,Setúbal,Sep-20 to Jan-21,33726.760091
4,R2 - AM Porto,Aveiro,Sep-19 to Feb-20,6041.956609


In [19]:
# Evaluate Distric County Population
population_density_district_county = population_density_censos_dicofres_renamed[["Concelho", "População"]].copy()
population_density_district_county = population_density_district_county.groupby(["Concelho"]).sum().reset_index(drop=False)
population_density_district_county.head()

,Concelho,População
0,Abrantes,39325
1,Aguiar da Beira,5473
2,Alandroal,5843
3,Albergaria-a-Velha,25252
4,Albufeira,40828


In [24]:
# Evaluate Distric Population
population_density_district = population_density_censos_dicofres_renamed[["Distrito", "População"]].copy()
population_density_district = population_density_district.groupby(["Distrito"]).sum().reset_index(drop=False)
population_density_district.head()

,Distrito,População
0,Aveiro,714197
1,Beja,152758
2,Braga,848185
3,Bragança,136252
4,Castelo Branco,196264


In [21]:
# Rename columns for merge (county)
population_density_district_county_renamed = population_density_district_county.copy()
population_density_district_county_renamed.rename(columns={"Concelho":"County_of_Origin"}, inplace=True)
population_density_district_county_renamed.head()

,County_of_Origin,População
0,Abrantes,39325
1,Aguiar da Beira,5473
2,Alandroal,5843
3,Albergaria-a-Velha,25252
4,Albufeira,40828


In [25]:
# Rename columns for merge (district)
population_density_district_renamed = population_density_district.copy()
population_density_district_renamed.rename(columns={"Distrito":"District_of_Origin"}, inplace=True)
population_density_district_renamed.head()

,District_of_Origin,População
0,Aveiro,714197
1,Beja,152758
2,Braga,848185
3,Bragança,136252
4,Castelo Branco,196264


In [26]:
# Merge county-district data with churn data
churn_users_grouped_county_pop = churn_users_grouped.copy().merge(
    population_density_district_county_renamed,
    on=["County_of_Origin"],
    how="left"
)

# Debugging
print(churn_users_grouped_county_pop.shape)
churn_users_grouped_county_pop.head()

(67, 6)


,Region_of_Origin,District_of_Origin,County_of_Origin,Period,Average_BusUsers_per_Day,População
0,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,8474.874368,175136
1,R1 - AM Lisboa,Lisboa,Amadora,Sep-20 to Jan-21,6985.896633,175136
2,R1 - AM Lisboa,Lisboa,Cascais,Sep-19 to Feb-20,13023.810109,206479
3,R1 - AM Lisboa,Lisboa,Cascais,Sep-20 to Jan-21,6693.154115,206479
4,R1 - AM Lisboa,Lisboa,Lisboa,Sep-19 to Feb-20,27874.289765,552700


In [27]:
# Merge district data with churn data
churn_users_grouped_district_pop = churn_users_grouped_district.copy().merge(
    population_density_district_renamed,
    on=["District_of_Origin"],
    how="left"
)

# Debugging
print(churn_users_grouped_district_pop.shape)
churn_users_grouped_district_pop.head()

(8, 5)


,Region_of_Origin,District_of_Origin,Period,Average_BusUsers_per_Day,População
0,R1 - AM Lisboa,Lisboa,Sep-19 to Feb-20,122085.933172,2250533
1,R1 - AM Lisboa,Lisboa,Sep-20 to Jan-21,81057.911693,2250533
2,R1 - AM Lisboa,Setúbal,Sep-19 to Feb-20,41852.012153,851258
3,R1 - AM Lisboa,Setúbal,Sep-20 to Jan-21,33726.760091,851258
4,R2 - AM Porto,Aveiro,Sep-19 to Feb-20,6041.956609,714197


In [28]:
# We may also want to evaluate special segments of the original dataset
churn_users_pf_pop_den_county_district = churn_users_pf.copy().merge(
    population_density_district_county_renamed,
    on=["County_of_Origin"],
    how="left"
)

# Debuggin
print(churn_users_pf_pop_den_county_district.shape)
churn_users_pf_pop_den_county_district.head()

(544, 8)


,Region_of_Origin,District_of_Origin,County_of_Origin,Period,GenderDescription,AgeClassDescription,Average_BusUsers_per_Day,População
0,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,15-24,294.194206,175136
1,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,25-34,1081.652817,175136
2,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,35-44,235.836653,175136
3,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,45-54,840.951323,175136
4,R1 - AM Lisboa,Lisboa,Amadora,Sep-19 to Feb-20,Female,55-64,1216.148092,175136
